### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [2]:
movies.head(3)

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [3]:
reviews.head(3)

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [4]:
reviews[reviews.movie_id==439]

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
129647,9970,439,7,1414365165,2014-10-26 23:12:45,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
266215,20177,439,5,1424384251,2015-02-19 22:17:31,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
344974,26184,439,8,1529809149,2018-06-24 02:59:09,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
634812,47839,439,7,1414595407,2014-10-29 15:10:07,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [21]:
avg_rating = reviews.groupby('movie_id')['rating'].mean()
review_size = reviews.groupby('movie_id').size()
recent_date = reviews.groupby('movie_id')['date'].max()
ranked_movies_df = pd.concat([avg_rating, review_size, recent_date], axis=1)
ranked_movies_df.columns = ['avg_rating','review_counts','latest_review_date']
ranked_movies_df = ranked_movies_df[ranked_movies_df['review_counts']>= 5]
ranked_movies_df = ranked_movies_df.sort_values(by=['avg_rating','review_counts','latest_review_date'], ascending=False)
pd.concat([ranked_movies_df, movies.set_index('movie_id')], join='inner', axis=1).reset_index()

,movie_id,avg_rating,review_counts,latest_review_date,movie,genre,date,1800's,1900's,2000's,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,4921860,10.000000,48,2016-08-14 17:16:50,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy,2015,0,0,1,...,1,0,0,0,0,0,1,0,0,0
1,5262972,10.000000,28,2016-01-08 00:44:43,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,...,0,0,0,0,0,0,1,1,0,0
2,5688932,10.000000,14,2018-06-17 01:44:48,Sorry to Bother You (2018),Comedy|Fantasy|Sci-Fi,2018,0,0,1,...,1,0,0,0,0,0,1,0,0,0
3,2737018,10.000000,10,2015-05-10 22:56:01,Selam (2013),Drama|Romance,2013,0,0,1,...,0,1,0,0,0,0,0,0,0,0
4,2560840,10.000000,6,2016-01-23 00:30:44,"Quiet Riot: Well Now You're Here, There's No W...",Documentary|Music,2014,0,0,1,...,0,0,0,0,1,0,0,0,0,0
5,2219210,10.000000,6,2013-07-22 23:30:52,Crawl Bitch Crawl (2012),Horror|Sci-Fi|Thriller,2012,0,0,1,...,0,0,0,0,0,0,0,0,0,1
6,4448444,10.000000,5,2017-09-09 13:51:48,Make Like a Dog (2015),Short|Comedy|Drama,2015,0,0,1,...,0,0,0,0,0,0,1,1,0,0
7,5131914,10.000000,5,2016-04-04 05:52:22,Pandorica (2016),Sci-Fi,2016,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8,2059318,10.000000,5,2014-02-13 15:48:31,Third Contact (2011),Mystery|Sci-Fi|Thriller,2011,0,0,1,...,0,0,0,0,0,0,0,0,0,1
9,1431149,10.000000,5,2013-07-26 17:19:01,Romeo Juliet (2009),Drama,2009,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [36]:
def create_ranked_movies(movies, reviews):
    '''
    Returns a dataframe filtered and sorted based on the following criteria:
        A movie with the highest average rating is considered best
        With ties, movies that have more ratings are better
        A movie must have a minimum of 5 ratings to be considered among the best movies
        If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating
    '''
    avg_rating = reviews.groupby('movie_id')['rating'].mean()
    review_size = reviews.groupby('movie_id').size()
    recent_date = reviews.groupby('movie_id')['date'].max()
    ranked_movies_df = pd.concat([avg_rating, review_size, recent_date], axis=1)
    ranked_movies_df.columns = ['avg_rating','review_counts','latest_review_date']
    ranked_movies_df = ranked_movies_df[ranked_movies_df['review_counts']>= 5]
    ranked_movies_df = ranked_movies_df.sort_values(by=['avg_rating','review_counts','latest_review_date'], ascending=False)
    
    return pd.concat([ranked_movies_df, movies.set_index('movie_id')], join='inner', axis=1).reset_index()

def popular_recommendations(user_id, n_top):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    
    return ranked_movies.loc[:n_top-1]['movie'].tolist()

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [ ]:
ranked_movies = create_ranked_movies(movies, reviews)

In [37]:
# Put your solutions for each of the cases here
# Top 20 movies recommended for id 1

recs_20_for_1 = popular_recommendations(1, 20)

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations(53968, 5)

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations(70000, 100)

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations(43, 35)

In [39]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [54]:
genres=['History','Short']
years=['2015', '2016', '2017', '2018']
n_top=5

In [60]:
movies_filtered = ranked_movies.copy()
if genres:
    genres_slctd = (movies_filtered[genres] == 1).any(axis=1)  # returns series
    movies_filtered = movies_filtered[genres_slctd]
if years:
    movies_filtered = movies_filtered[movies_filtered['date'].isin(years)]
# movies_filtered.reset_index(drop=True)
movies_filtered.reset_index(drop=True).loc[:n_top-1]

,movie_id,avg_rating,review_counts,latest_review_date,movie,genre,date,1800's,1900's,2000's,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,5262972,10.0,28,2016-01-08 00:44:43,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,...,0,0,0,0,0,0,1,1,0,0
1,4448444,10.0,5,2017-09-09 13:51:48,Make Like a Dog (2015),Short|Comedy|Drama,2015,0,0,1,...,0,0,0,0,0,0,1,1,0,0
2,6316138,9.2,35,2018-06-22 22:19:09,Ayla: The Daughter of War (2017),Drama|History|War,2017,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,5098836,9.1,10,2018-04-27 22:13:39,I Believe in Miracles (2015),Documentary|History|Sport,2015,0,0,1,...,0,0,0,0,1,0,0,0,0,0
4,5807512,9.0,8,2017-04-18 22:31:27,Inner Workings (2016),Animation|Short|Family,2016,0,0,1,...,0,0,0,0,0,1,0,1,0,0


In [63]:
def popular_recs_filtered(user_id, n_top, years=None, genres=None):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    movies_filtered = ranked_movies.copy()
    if genres:
        genres_slctd = (movies_filtered[genres] == 1).any(axis=1)  # returns series
        movies_filtered = movies_filtered[genres_slctd]
    if years:
        movies_filtered = movies_filtered[movies_filtered['date'].isin(years)]
    top_movies = movies_filtered.reset_index(drop=True).loc[:n_top-1]['movie'].tolist()
    
    return top_movies

In [64]:
ranked_movies = create_ranked_movies(movies, reviews)
popular_recs_filtered('1', 20, years=['2015', '2016', '2017', '2018'], genres=['History'])

['Ayla: The Daughter of War (2017)',
 'I Believe in Miracles (2015)',
 'The Farthest (2017)',
 'Sado (2015)',
 'Hatred (2016)',
 'Kincsem (2017)',
 'Nise - O Coração da Loucura (2015)',
 'LA 92 (2017)',
 'Straight Outta Compton (2015)',
 'Manjhi: The Mountain Man (2015)',
 'Only the Dead (2015)',
 'Spotlight (2015)',
 'Under sandet (2015)',
 'Airlift (2016)',
 'Dunkirk (2017)',
 'Taeksi Woonjunsa (2017)',
 'The Battleship Island (2017)',
 'Darkest Hour (2017)',
 'Best of Enemies (2015)',
 'The Ghazi Attack (2017)']

In [66]:
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])

['Ayla: The Daughter of War (2017)',
 'I Believe in Miracles (2015)',
 'The Farthest (2017)',
 'Sado (2015)',
 'Hatred (2016)',
 'Kincsem (2017)',
 'Nise - O Coração da Loucura (2015)',
 'LA 92 (2017)',
 'Straight Outta Compton (2015)',
 'Manjhi: The Mountain Man (2015)',
 'Only the Dead (2015)',
 'Spotlight (2015)',
 'Under sandet (2015)',
 'Airlift (2016)',
 'Dunkirk (2017)',
 'Taeksi Woonjunsa (2017)',
 'The Battleship Island (2017)',
 'Darkest Hour (2017)',
 'Best of Enemies (2015)',
 'The Ghazi Attack (2017)']